In [1]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as N
import matplotlib.pyplot as plt

In [2]:
import torch
torch.manual_seed(42)

batch_size = 32
epochs = 5
learning_rate = 1e-3

In [3]:
training_data = datasets.MNIST(
    root= '../pytorch-basics/datasets/mnist/train',
    train= True,
    download= True,
    transform= ToTensor()
)

test_data = datasets.MNIST(
    root= '../pytorch-basics/datasets/mnist/test',
    train= False,
    download= True,
    transform= ToTensor()
)

In [4]:
train_dataloader = DataLoader(training_data, batch_size= batch_size, shuffle= True)
test_dataloader = DataLoader(test_data, batch_size= batch_size, shuffle= True) 

In [5]:
model = N.Sequential(
            N.Conv2d(in_channels= 1, out_channels= 16, kernel_size= 3, stride= 1, padding= 1),
            N.Tanh(),
            N.MaxPool2d(kernel_size= 2, stride= 2),

            N.Conv2d(in_channels= 16, out_channels= 32, kernel_size= 3, stride= 1, padding= 1),
            N.Tanh(),
            N.MaxPool2d(kernel_size= 2, stride= 2),

            N.Flatten(),

            N.Linear(in_features= 32*7*7, out_features= 500),
            N.Tanh(),
            
            N.Linear(in_features= 500, out_features= 100),
            N.Tanh(),
            
            N.Linear(in_features= 100, out_features= 50),
            N.Tanh(),
            
            N.Linear(in_features= 50, out_features= 10),
)

In [6]:
from torchinfo import summary

summary(model, input_size= (batch_size, 1, 28, 28), col_names=['input_size', 'output_size', 'num_params', 'trainable'], 

        row_settings=['var_names'], verbose=0)

Layer (type (var_name))                  Input Shape               Output Shape              Param #                   Trainable
Sequential (Sequential)                  [32, 1, 28, 28]           [32, 10]                  --                        True
├─Conv2d (0)                             [32, 1, 28, 28]           [32, 16, 28, 28]          160                       True
├─Tanh (1)                               [32, 16, 28, 28]          [32, 16, 28, 28]          --                        --
├─MaxPool2d (2)                          [32, 16, 28, 28]          [32, 16, 14, 14]          --                        --
├─Conv2d (3)                             [32, 16, 14, 14]          [32, 32, 14, 14]          4,640                     True
├─Tanh (4)                               [32, 32, 14, 14]          [32, 32, 14, 14]          --                        --
├─MaxPool2d (5)                          [32, 32, 14, 14]          [32, 32, 7, 7]            --                        --
├─Flatten (

In [7]:
from torchmetrics import Accuracy

loss_fn = N.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
accuracy_fn = Accuracy(task= 'multiclass', num_classes= 10).cuda()

In [8]:
for _ in range(epochs):
    train_loss = 0
    print(f"\nStarting epoch: {_ + 1}\n~~~~~~~~~~~")
    for index, (input, target) in enumerate(train_dataloader):
        input, target = input.cuda(), target.cuda()
        model.train()
        output = model(input)

        loss = loss_fn(output, target)
        train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(index % 500 == 0):
            print(f"Batch: {index} Loss:{loss:0.5f}")
    
    train_loss /= len(train_dataloader)
    test_loss, test_acc = 0, 0
    model.eval()

    with torch.inference_mode():
        for test_input, test_target in test_dataloader:
            test_input, test_target = test_input.cuda(), test_target.cuda()

            test_pred = model(test_input)
            test_loss += loss_fn(test_pred, test_target)
            test_acc += accuracy_fn(test_target, test_pred.argmax(dim= 1))
        
        test_loss /= len(test_dataloader)
        test_acc /= len(test_dataloader)

    print(f"Train loss: {train_loss:0.5f} Test loss: {test_loss:0.5f} Test acc: {test_acc*100:0.2f}%")


Starting epoch: 1
~~~~~~~~~~~
Batch: 0 Loss:2.29484
Batch: 500 Loss:0.19320
Batch: 1000 Loss:0.04784
Batch: 1500 Loss:0.22656
Train loss: 0.19112 Test loss: 0.07889 Test acc: 97.67%

Starting epoch: 2
~~~~~~~~~~~
Batch: 0 Loss:0.04243
Batch: 500 Loss:0.13195
Batch: 1000 Loss:0.01587
Batch: 1500 Loss:0.00416
Train loss: 0.06111 Test loss: 0.06028 Test acc: 98.26%

Starting epoch: 3
~~~~~~~~~~~
Batch: 0 Loss:0.01905
Batch: 500 Loss:0.01242
Batch: 1000 Loss:0.00604
Batch: 1500 Loss:0.00730
Train loss: 0.04295 Test loss: 0.04545 Test acc: 98.45%

Starting epoch: 4
~~~~~~~~~~~
Batch: 0 Loss:0.01236
Batch: 500 Loss:0.02852
Batch: 1000 Loss:0.00185
Batch: 1500 Loss:0.00985
Train loss: 0.03309 Test loss: 0.04948 Test acc: 98.41%

Starting epoch: 5
~~~~~~~~~~~
Batch: 0 Loss:0.05067
Batch: 500 Loss:0.00071
Batch: 1000 Loss:0.03898
Batch: 1500 Loss:0.00204
Train loss: 0.02870 Test loss: 0.04317 Test acc: 98.69%
